In [95]:
import pandas as pd
import json
import re
import unicodedata

# loading json file
with open('faculty_finder/faculty2.json', 'r',encoding='utf-8') as f:
    data = json.load(f)
    
df=pd.DataFrame(data)

In [96]:
df

,name,education,phone,address,email,specializations,biography,teaching,publications,research,website_links
0,Yash vasavada,"PhD (Electrical Engineering), Virginia Polytec...",079-68261634,"# 1224, FB-1, DA-IICT, Gandhinagar, Gujarat, I...",yash_vasavada[at]dau[dot]ac[dot]in,"Communication, Signal Processing, Machine Lear...",Yash Vasavada is currently a Professor at DAII...,"[Introduction to Communication Systems, Advanc...","[, Michael Parr, Nidhi Sindhav, and Saumi S., ...","Communication, Signal Processing, Machine Lear...",[http://intranet.daiict.ac.in/~yash_vasavada/i...
1,Yash agrawal,"PhD (Electronics & Communication), NIT Hamirpur","079-68261629, 9882114669","# 1101, FB-1, DA-IICT, Gandhinagar, Gujarat, I...","yash_agrawal[at]dau[dot]ac[dot]in, mr[dot]yash...","VLSI, Nanotechnology, Numerical Method Techniq...",Dr. Yash Agrawal received his Ph.D. and M.Tech...,"[Digital Logic Design, Digital Design using HD...","[Y. Agrawal, M. Girish, and R. Chandel, “An ef...","VLSI, Nanotechnology, Numerical Method Techniq...",[http://www.linkedin.com/in/yash-agrawal-525b0...
2,Vinay palaparthy,"PhD (Electrical Engineering), IIT Bombay",079-68261677,"# 1208, FB-1, DA-IICT, Gandhinagar, Gujarat, I...",vinay_shrinivas[at]dau[dot]ac[dot]in,"Micro-Electro-Mechanical Systems (MEMS), Physi...",,"[Introduction to MEMS, Sensors and Instrumenta...",[],"Micro-Electro-Mechanical Systems (MEMS), Physi...",[]
3,Tathagata bandyopadhyay,"PhD (Statistics), University of Calcutta, Kolkata",079-68261572,,tathagata_b[at]dau[dot]ac[dot]in,"Statistical Inference, Survey Sampling, Discre...",Dr. Tathagata Bandyopadhyay has been in the ac...,"[Research Methodology, Data Analytics, Bayesia...","[Arindam Chatterjee,, , Ayoushman Bhattacharya...","Statistical Inference, Survey Sampling, Discre...",[]
4,Tapas kumar maiti,PhD (Electronics & Telecommunication Engineeri...,079-68261637,"# 2206, FB-2, DA-IICT, Gandhinagar, Gujarat, I...",tapas_kumar[at]dau[dot]ac[dot]in,,"Dr. Tapas Kumar Maiti, a former associate prof...","[IE416: Robot Programming (Winter 2024), IE410...",[],,"[https://sites.google.com/view/cybernoid, http..."
...,...,...,...,...,...,...,...,...,...,...,...
62,Ajay beniwal,PhD (Electronics and Communication Engineering...,079-68261745,"# 3211, FB-3, DA-IICT, Gandhinagar, Gujarat, I...",ajay_beniwal[at]dau[dot]ac[dot]in,Flexible and Printable Electronics for Healthc...,Dr. Ajay Beniwal is currently serving as an As...,[],"[, Priyanka Ganguly, Gaurav Khandelwal, Rahul ...",Flexible and Printable Electronics for Healthc...,[https://sites.google.com/view/ajay-beniwal/home]
63,Aditya tatu,"PhD (Computer Science), University of Copenhag...",079-68261540,"#1206, FB-1, DA-IICT, Gandhinagar, Gujarat, In...",aditya_tatu[at]dau[dot]ac[dot]in,"Computer Vision, Image Processing, Pattern Rec...",Aditya Tatu received his Phd from the Departme...,"[Linear Algebra & Functional Analysis, Optimiz...","[Darshan Batavia and Aditya Tatu, Estimating g...","Computer Vision, Image Processing, Pattern Rec...",[]
64,Abhishek tilva,"PhD (Statistics), Columbia University, New Yor...",079-68261549,"# 2209, FB-2, DA-IICT, Gandhinagar, Gujarat, I...",abhishek_tilva[at]dau[dot]ac[dot]in,"Arbitrage Theory, Stochastic Portfolio Theory,...",Abhishek Tilva currently serves as an Assistan...,[],"[Bayraktar, Erhan, Donghan Kim, and, . ""Quanti...","Arbitrage Theory, Stochastic Portfolio Theory,...",[https://scholar.google.com/citations?user=1YC...
65,Abhishek jindal,"PhD (Electronics & Communication Engineering),...",079-68261654,"# 4101, FB-4, DA-IICT, Gandhinagar, Gujarat, I...",abhishek_jindal[at]dau[dot]ac[dot]in,"Reinforcement Learning, Deep Learning for Fina...",I received the B. Tech. and M. Tech. degrees f...,"[Reinforcement Learning, Applications of Machi...",[],"Reinforcement Learning, Deep Learning for Fina...",[https://scholar.google.co.in/citations?user=J...


In [97]:
def title_case_name(name):
    if not isinstance(name, str) or name.strip() == "":
        return None
    return name.title()

df['name'] = df['name'].apply(title_case_name)

In [98]:
def clean_email(email):
    if not isinstance(email, str) or not email.strip():
        return None

    email = email.replace("[at]", "@").replace("[dot]", ".")
    email = email.replace(" ", "")

    emails = [e for e in email.split(",") if "@" in e]

    return emails if emails else None

df['email'] = df['email'].apply(clean_email)

In [99]:

def clean_text(text):
    if not isinstance(text, str) or not text.strip():
        return None

    # Normalizing unicode (fixes most encoding issues)
    text = unicodedata.normalize("NFKC", text)

    # Remove non-printable characters
    text = "".join(ch for ch in text if ch.isprintable())

    # Normalizing whitespace
    text = re.sub(r"\s+", " ", text).strip()

    return text

df['education'] = df['education'].apply(clean_text)
df['address'] = df['address'].apply(clean_text)
df['specializations']= df['specializations'].apply(clean_text)
df['biography'] = df['biography'].apply(clean_text)
df['research']=df['research'].apply(clean_text)


In [100]:
def clean_and_categorize_phone(phone):
    if not isinstance(phone, str) or not phone.strip():
        return None
    
    # Remove hyphens and spaces, then extract digit groups
    phone_cleaned = phone.replace("-", "").replace(" ", "")
    result = {"mobile": [], "landline": []}
    
    for num in re.findall(r"\d+", phone_cleaned):
        if len(num) == 10 and num[0] in "6789":
            result["mobile"].append(num)
        elif num.startswith("0") and 10 <= len(num) <= 12:
            result["landline"].append(num)
    
    result = {k: v for k, v in result.items() if v}
    return result if result else None

df['phone'] = df['phone'].apply(clean_and_categorize_phone)

In [101]:
def clean_list(lst):
    if not isinstance(lst, list):
        return None

    cleaned = [
        clean_text(item)
        for item in lst
        if isinstance(item, str) and clean_text(item)
    ]

    return cleaned if cleaned else None

df['teaching'] = df['teaching'].apply(clean_list)

In [102]:
def clean_publications(pub_list):
    """Merging broken publication fragments"""
    if not isinstance(pub_list, list):
        return None

    publications = []
    buffer = ""

    for item in pub_list:
        if not isinstance(item, str):
            continue

        item = clean_text(item)
        if not item:
            continue

        buffer += " " + item

        if re.search(r"\b(19|20)\d{2}\b", buffer) or "doi" in buffer.lower():
            publications.append(buffer.strip())
            buffer = ""

    if buffer.strip():
        publications.append(buffer.strip())

    return publications if publications else None

df["publications"] = df["publications"].apply(clean_publications)

In [103]:
def clean_links(links):
    if not isinstance(links, list):
        return None

    links = [l.strip() for l in links if isinstance(l, str) and l.startswith("http")]
    return links if links else None

df["website_links"] = df["website_links"].apply(clean_links)

In [104]:
def categorize_links(links):
    if not isinstance(links, list) or not links:
        return None

    categorized = {
        "personal_website": [],
        "google_scholar": [],
        "linkedin": [],
        "youtube": [],
        "other": []
    }

    for link in links:
        if "scholar.google" in link:
            categorized["google_scholar"].append(link)

        elif "linkedin.com" in link:
            categorized["linkedin"].append(link)

        elif "youtube.com" in link or "youtu.be" in link:
            categorized["youtube"].append(link)

        elif "sites.google" in link or "github.io" in link:
            categorized["personal_website"].append(link)

        else:
            categorized["other"].append(link)

    categorized = {k: v for k, v in categorized.items() if v}

    return categorized if categorized else None

df["website_links"] = df["website_links"].apply(categorize_links)


In [105]:
# normalizing all values including empty strings to None
df = df.applymap(
    lambda x: None if isinstance(x, str) and not x.strip() else x
)
df = df.where(pd.notnull(df), None)

C:\Users\tcpladmin255\AppData\Local\Temp\ipykernel_13072\3487208760.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(


In [106]:
df.to_csv(
    "faculty_cleaned.csv",
    index=False
)

print("Cleaning completed and saved to faculty_cleaned.csv")

Cleaning completed and saved to faculty_cleaned.csv
